In [1]:
import os, sys
import pandas as pd
pd.options.display.max_columns = 100

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.feature import get_feat_d_from_ID
from util.genome import is_overlap
from util.operon import get_operon_ID_set

In [3]:
all_muts_df = pd.read_pickle("./data/2_3_df.pkl")
display(all_muts_df.shape, all_muts_df.head())

(5676, 30)

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links
0,9,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,1 124 1 1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']}
1,10,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,1 124 1 1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']}
2,11,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,,rph,NaN,1 124 1 1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079..."
3,12,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,1 124 1 1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']}
4,13,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,1 124 1 1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']}


In [4]:
TU_objects_df = pd.read_pickle("./data/TU_objects_df.pkl")
TU_objects_df["TU range"] = TU_objects_df.apply(lambda row: (row["TU_POSLEFT"], row["TU_POSRIGHT"]), axis=1)

TU_df = pd.read_pickle("./data/TU_df.pkl")

operon_df = pd.read_pickle("./data/operon_df.pkl",)
operon_df["range"] = operon_df.apply(lambda row: (row["REGULATIONPOSLEFT"], row["REGULATIONPOSRIGHT"]), axis=1)

In [5]:
TF_TU_mut_hit_df = TU_objects_df.copy()
TF_TU_mut_hit_df = TF_TU_mut_hit_df.drop(
    columns=['NUMTU',
             'TU_TYPE',
             'TU_OBJECT_ID',
             'TU_OBJECT_POSLEFT',
             'TU_OBJECT_POSRIGHT',
             'TU_OBJECT_STRAND',
             'TU_OBJECT_COLORCLASS',
             'TU_OBJECT_DESCRIPTION',
             'TU_OBJECT_SIGMA',
             'TU_OBJECT_EVIDENCE',
             'TU_OBJECT_RI_TYPE',
             'TU_OBJECT_TYPE',
             'EVIDENCE']
)

# filtering to only use TUs with regulator binding sites.
# I'm ultimately extracting the Regulon (all simple) according to the TFBS that is linked to that TU (through the "ST" TU_OBJECT_CLASS)
# There's an issue here though. If the TF for a gene doesn't have it's TU regulated by this TF, then mutations to the Regulon's TF TU won't get linked to the regulon (ASW-1857.
TF_TU_mut_hit_df = TF_TU_mut_hit_df[TF_TU_mut_hit_df["TU_OBJECT_CLASS"]=="ST"]

TF_TU_mut_hit_df = TF_TU_mut_hit_df.drop_duplicates()
TF_TU_mut_hit_df.head()

,TRANSCRIPTION_UNIT_ID,TU_POSLEFT,TU_POSRIGHT,TU_OBJECT_CLASS,TU_OBJECT_NAME,TU range
12,ECK120008914,1825955,1832327,ST,ArgR,"(1825955, 1832327)"
15,ECK120008914,1825955,1832327,ST,NtrC,"(1825955, 1832327)"
52,ECK120008916,2800586,2804461,ST,NrdR,"(2800586, 2804461)"
53,ECK120008916,2800586,2804461,ST,IscR,"(2800586, 2804461)"
54,ECK120008916,2800586,2804461,ST,Fur,"(2800586, 2804461)"


In [6]:
def get_TU_ID_set(feat_ID):
    return set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"]==feat_ID]["TRANSCRIPTION_UNIT_ID"])


def get_complex_regs_as_single_reg_set(reg_str):
    clean_reg_name_set = set()
    if ',' in reg_str:
        clean_reg_name_set |= set(reg_str.split(','))
    else:
        clean_reg_name_set |= set([reg_str])
    return clean_reg_name_set


def get_reg_name_set(TU_ID):
    df = TU_objects_df[TU_objects_df["TRANSCRIPTION_UNIT_ID"]==TU_ID]
    regulator_df = df[df["TU_OBJECT_CLASS"]=="ST"]
    reg_name_set = set()
    for reg_str in regulator_df["TU_OBJECT_NAME"]:
        reg_name_set |= get_complex_regs_as_single_reg_set(reg_str)
    return reg_name_set


# Must split up regulonDB complex regulatons into single regulators.
assert get_reg_name_set("ECK120009661") == {'NagC', 'CRP', 'Mlc'}  # Contains the complex regulator "NagC,Mlc".


def _get_genetic_target_d_from_ID(gen_targ_ID, mut_row):
    gen_targ_d = dict()
    for d in mut_row["genetic features"]:
        if d["RegulonDB ID"] == gen_targ_ID:
            gen_targ_d = d
            break
    return gen_targ_d


def get_regulators(mut_row, TF_TU_mut_hit_df):
    annot_json = []
    for feat_d in mut_row["genomic features"]:
        # !!! This part only handles mutations that land within a genomic feature
        # This used to have the condition '"RegulonDB ID" in feat_d' though I don't feel this is necessary
        # since features should now always have RegulonDB IDs. If they don't, this should crash to indicate the fact.
        if '/' not in feat_d["RegulonDB ID"]:

            # This will allow for multiple entries of the same regulon being annotated for a mutation
            # the same Regulon mutated multiple times by a mutation affecting
            # overlapping features or by a mutated feature servicing multiple TUs like in the case of the gcvA gcvB TFBS and the GcvA regulon.            
            for TU in get_TU_ID_set(feat_d["RegulonDB ID"]):  # It's fine to keep the TUs as a set since we don't expect to have an individual mutated feature affect the same TU twice.:
                for r in get_reg_name_set(TU):
                    annot_json.append({"name": r})

        # This will catch whether genomic features are integenic regions.
        # Get regulon genetic feature link dict through overlapping mutation and regulon ranges.
        # All genetic features are annotated within both the "genetic features" and "genetic feature links" column.
        # If no integenic genomic feature is hit by a mutation, use the genetic feature range in an overlap analysis with operons.
        else:
            TF_TU_mut_hit_df["mutation hit"] = TF_TU_mut_hit_df["TU range"].apply(is_overlap, args=[feat_d["range"]])
            df = TF_TU_mut_hit_df[TF_TU_mut_hit_df["mutation hit"]]
            for _, TF_TU in df.iterrows():            
                for reg in get_complex_regs_as_single_reg_set(TF_TU["TU_OBJECT_NAME"]):
                    annot_json.append({"name": reg})
            TF_TU_mut_hit_df = TF_TU_mut_hit_df.drop(columns=["mutation hit"])  # TODO: This isn't necessary. Should instead do what had done for imod (though first setup unit test to ensure expected behavior isn't broken)
    return annot_json


all_muts_df["regulators"] = all_muts_df.apply(lambda r: get_regulators(r, TF_TU_mut_hit_df), axis=1)
all_muts_df.head()

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators
0,9,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,1 124 1 1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'PhoP'}, {'name': 'NagC'}, {'name': ..."
1,10,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,1 124 1 1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[]
2,11,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,,rph,NaN,1 124 1 1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[]
3,12,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,1 124 1 1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[]
4,13,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,1 124 1 1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}]


In [7]:
# TODO: There's currently a function with the same name within util.operon. Refactor and combine the two
# def get_operon_name_set(TU_ID):
#     op_name_set = set()
#     op_IDs = TU_df[TU_df["TRANSCRIPTION_UNIT_ID"]==TU_ID]["OPERON_ID"].unique()
#     for op_ID in op_IDs:
#         op_name_set.add(operon_df[operon_df["OPERON_ID"]==op_ID].iloc[0]["OPERON_NAME"])  # Assuming that each operon only has one name
#     return op_name_set


def _get_operon_ID_set(TU_id):
    return set(TU_df[TU_df["TRANSCRIPTION_UNIT_ID"]==TU_id]["OPERON_ID"].unique())


# Very similar to get_regulators.
def get_regulator_operon_links_d(mut_row, TU_objects_df):
    reg_op_link_d = dict()
    for gen_targ_ID, feat_ID_set in mut_row["genetic feature links"].items():  # Using the genomic feature IDs since they are unique and not duplicated like with the "genomic features" column
        for feat_ID in feat_ID_set:
            feat_d = get_feat_d_from_ID(feat_ID, mut_row)

            # This part only handles mutations that land within a genomic feature.
            if '/' not in feat_d["RegulonDB ID"]:
#                 operon_name_set = set()
                op_id_set = set()
                reg_name_set = set()
                TU_id_mut_hit_set = get_TU_ID_set(feat_d["RegulonDB ID"])
                for TU_id in TU_id_mut_hit_set:
                    reg_name_set |= get_reg_name_set(TU_id)
                    op_id_set |= _get_operon_ID_set(TU_id)
                for reg_name in reg_name_set:
                    if reg_name not in reg_op_link_d.keys(): reg_op_link_d[reg_name] = []
                    reg_op_link_d[reg_name] += list(op_id_set)
#                     operon_name_set |= get_operon_name_set(tu_id)  # using a set because only want one operon per mutated feature TU.
#                 for reg_name in reg_name_set:
#                     if reg_name not in reg_op_link_d.keys(): reg_op_link_d[reg_name] = []
#                     reg_op_link_d[reg_name] += list(operon_name_set)


            # This code is only meant to serve intergenic regions.
            # Get operon genetic feature link dict through overlapping mutated feature and operon ranges.
            # All genetic features are annotated within both the "genetic features" and "genetic feature links" column.
            else:
                TU_objects_df["mutation hit"] = TU_objects_df["TU range"].apply(is_overlap, args=[feat_d["range"]])
                TU_mut_hit_df = TU_objects_df[TU_objects_df["mutation hit"]]
                TU_id_mut_hit_set = set(TU_mut_hit_df["TRANSCRIPTION_UNIT_ID"])
#                 operon_name_set = set()
                op_id_set = set()
                reg_name_set = set()
                for TU_id in TU_id_mut_hit_set:
                    reg_name_set |= get_reg_name_set(TU_id)
                    op_id_set |= _get_operon_ID_set(TU_id)
#                     operon_name_set |= get_operon_name_set(TU_id)  # using a set because only want one operon per mutated feature TU.
                for reg_name in reg_name_set:
                    if reg_name not in reg_op_link_d.keys(): reg_op_link_d[reg_name] = []
                    reg_op_link_d[reg_name] += list(op_id_set)
#                     reg_op_link_d[reg_name] += list(operon_name_set)
                # cleanup, TODO: This isn't necessary. Should instead do what had done for imod (though first setup unit test to ensure expected behavior isn't broken)
                if "mutation hit" in TU_objects_df.columns.values:
                    TU_objects_df = TU_objects_df.drop(columns=["mutation hit"])

    return reg_op_link_d


all_muts_df["regulator links"] = all_muts_df.apply(lambda r: get_regulator_operon_links_d(r, TU_objects_df), axis=1)
all_muts_df.head()

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links
0,9,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,1 124 1 1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'PhoP'}, {'name': 'NagC'}, {'name': ...","{'NagC': ['ECK125162049'], 'PhoP': ['ECK125162..."
1,10,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,1 124 1 1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{}
2,11,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,,rph,NaN,1 124 1 1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{}
3,12,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,1 124 1 1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{}
4,13,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,1 124 1 1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['ECK125147410']}


In [8]:
all_muts_df[all_muts_df["mutation target annotation"]=="yhcC/gltB"]

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links
1013,4149,JCL260__isobutanol_production_host-M9-MG1655gbk,1,999,1,1,1.0,3352254,SUB,2 bp→GC,intergenic (‑569/‑12),yhcC/gltB,NC_000913,1 999 1 1,False,"(3352254, 3352255)",{ECK120000058},"[{'name': 'arcB', 'RegulonDB ID': 'ECK12000005...",False,False,{},{},{},{},{},False,"[{'name': 'arcB', 'RegulonDB ID': 'ECK12000005...",{'ECK120000058': ['ECK120000058']},"[{'name': 'arcB', 'RegulonDB ID': 'ECK12001512...",{'ECK120015127': ['ECK120000058']},[],{}
1237,4372,JCL260__isobutanol_production_host-M9-MG1655gbk,2,999,1,1,1.0,3352254,SUB,2 bp→GC,intergenic (‑569/‑12),yhcC/gltB,NC_000913,2 999 1 1,False,"(3352254, 3352255)",{ECK120000058},"[{'name': 'arcB', 'RegulonDB ID': 'ECK12000005...",False,False,{},{},{},{},{},False,"[{'name': 'arcB', 'RegulonDB ID': 'ECK12000005...",{'ECK120000058': ['ECK120000058']},"[{'name': 'arcB', 'RegulonDB ID': 'ECK12001512...",{'ECK120015127': ['ECK120000058']},[],{}
2250,3479,SSW_GLU_XYL,1,163,7,1,1.0,3354228,INS,(T)7→8,intergenic (‑178/‑497),yhcC/gltB,NaN,1 163 7 1,False,"(3354228, 3354228)",{},[],False,False,{},"{ECK125137697, ECK125137696}",{},{},{},False,"[{'name': 'yhcCp6', 'RegulonDB ID': 'ECK125137...",{},[],{},[],{}
4555,14691,pgiPAE,9,18,1,1,1.0,3354297,SNP,G→T,intergenic (‑247/‑428),yhcC/gltB,NC_000913_3_pae_pgi,9 18 1 1,False,"(3354297, 3354297)",{},[],False,False,{ECK120013884},{},{},{},{},False,"[{'name': 'gltBDF TFBS', 'RegulonDB ID': 'ECK1...",{},"[{'name': 'gltBDF', 'RegulonDB ID': 'ECK120014...",{},"[{'name': 'FNR'}, {'name': 'Nac'}, {'name': 'A...",{}


In [9]:
all_muts_df.to_pickle("./data/2_4_df.pkl")